## Assignment 1: Multilingual Embedding-based Machine Translation 

## 1 Task) 13 points

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

# PLEASE CHECK ASSERT, BE SURE THAT YOU IMPLEMENTED ALL THE CODE CORRECT IN FIRST PART

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Fragment of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our purposes.

## Data

In [2]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://www.dropbox.com/scl/fo/czrdhfndzvirs0c3x4adh/AIDkoaHAyd3bcpAegJ4bz3A?rlkey=f7xnl9lqiahjbor3ucxyy6u6p&st=ee70k4ik&dl=0)
* [cc.ru.300.vec.zip](https://www.dropbox.com/scl/fo/czrdhfndzvirs0c3x4adh/AIDkoaHAyd3bcpAegJ4bz3A?rlkey=f7xnl9lqiahjbor3ucxyy6u6p&st=ee70k4ik&dl=0)

Load embeddings for ukrainian and russian.

In [3]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [4]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [5]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [53]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [8]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [9]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [20]:
# Load word pairs from training file
pairs = []
try:
    with open(r"C:\Users\Admin\Desktop\sabaq(C3T2)\C3T2\NLP\Assignment 1\ukr_rus.train.txt", "r", encoding="utf-8") as file:
        for line in file:
            uk_word, ru_word = line.strip().split()
            pairs.append((uk_word, ru_word))
except FileNotFoundError:
    print("Error: Training file 'ukr_rus.train.txt' not found. Check your file path.")

# Debug loaded pairs
print(f"Loaded {len(pairs)} pairs.")
print(f"First 5 pairs: {pairs[:5]}")


Loaded 1927 pairs.
First 5 pairs: [('iмовірно', 'вероятно'), ('iснує', 'существует'), ('iспит', 'экзамен'), ('абияк', 'как-нибудь'), ('або', 'или')]


In [33]:
# Filter pairs to ensure both words have embeddings
pairs = [(uk, ru) for uk, ru in pairs if uk in uk_emb and ru in ru_emb]

# Debugging length of pairs
print(f"Filtered pairs: {len(pairs)}")

Filtered pairs: 1840


In [34]:
# Initialize test matrices
X_test = []
Y_test = []

# Build X_test and Y_test matrices
for uk_word, ru_word in pairs:
    if uk_word in uk_emb and ru_word in ru_emb:
        X_test.append(uk_emb[uk_word])
        Y_test.append(ru_emb[ru_word])

# Convert to numpy arrays
X_test = np.array(X_test)
Y_test = np.array(Y_test)

# Debugging shapes
print(f"X_test shape: {X_test.shape}")
print(f"Y_test shape: {Y_test.shape}")

X_test shape: (1840, 300)
Y_test shape: (1840, 300)


In [35]:
from sklearn.linear_model import LinearRegression

# Initialize and train the linear regression model
mapping = LinearRegression()
mapping.fit(X_train, Y_train)

# Debugging coefficients
print(f"Mapping coefficients shape: {mapping.coef_.shape}")

Mapping coefficients shape: (300, 300)


In [36]:
mapped_vectors = mapping.predict(X_test)
print(f"Mapped vectors shape: {mapped_vectors.shape}")

Mapped vectors shape: (1840, 300)


Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [23]:
# Example: Map Ukrainian word "серпень" to Russian
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
print(ru_emb.most_similar(august, topn=5))  # Get top-5 similar Russian words

[('апрель', 0.8541591763496399), ('июнь', 0.8411963582038879), ('март', 0.8397400379180908), ('сентябрь', 0.8359215259552002), ('февраль', 0.8328747749328613)]


We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [47]:
from scipy.spatial.distance import cdist
import numpy as np

def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0

    # Extract the embeddings of the Russian words in the pairs
    ru_words = [ru for _, ru in pairs]
    ru_embeddings = np.array([ru_emb[word] for word in ru_words if word in ru_emb])

    # Compute cosine similarities
    distances = cdist(mapped_vectors, ru_embeddings, metric="cosine")

    # Find nearest neighbors and check if the correct Russian word is within the top 'topn'
    for i, (_, ru) in enumerate(pairs):
        if ru not in ru_emb:
            continue
        nearest_indices = np.argsort(distances[i])[:topn]
        nearest_words = [ru_words[idx] for idx in nearest_indices]
        if ru in nearest_words:
            num_matches += 1

    precision_val = num_matches / len(pairs)
    return precision_val

In [48]:
print(f"Number of pairs: {len(pairs)}")
print(f"Number of mapped vectors: {len(mapped_vectors)}")

Number of pairs: 1840
Number of mapped vectors: 1840


In [50]:
# Filter pairs to ensure both words have embeddings
filtered_pairs = [(uk, ru) for uk, ru in pairs if uk in uk_emb and ru in ru_emb]

# Prepare training data again based on filtered pairs
X_train = np.array([uk_emb[uk] for uk, ru in filtered_pairs])
Y_train = np.array([ru_emb[ru] for uk, ru in filtered_pairs])

# Verify shapes of training data
print(f"Filtered X_train shape: {X_train.shape}")
print(f"Filtered Y_train shape: {Y_train.shape}")

Filtered X_train shape: (1840, 300)
Filtered Y_train shape: (1840, 300)


In [51]:
# Map the filtered Ukrainian embeddings to Russian space
mapped_vectors = mapping.predict(X_train)

# Re-run precision calculation
precision_val = precision(filtered_pairs, mapped_vectors, topn=1)
print(f"Precision@1: {precision_val}")

Precision@1: 0.8429347826086957


In [54]:
# Correcting `august` to a 2D array for precision calculation
mapped_august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
assert precision([("серпень", "август")], mapped_august, topn=5) == 0.0
assert precision([("серпень", "август")], mapped_august, topn=9) == 1.0
assert precision([("серпень", "август")], mapped_august, topn=10) == 1.0

AssertionError: 

In [55]:
# Precision calculations with filtered pairs
mapped_test_vectors = mapping.predict(X_test)
assert precision(filtered_pairs, mapped_test_vectors, topn=1) == 0.0
assert precision(filtered_pairs, mapped_test_vectors, topn=10) == 1.0

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [58]:
# Run precision function on corrected test data
precision_top1 = precision(pairs, mapping.predict(X_test), topn=1)
precision_top5 = precision(pairs, mapping.predict(X_test), topn=5)

# Assert precision meets the requirements
assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

print(f"Precision@1: {precision_top1}")
print(f"Precision@5: {precision_top5}")

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [59]:
def learn_transform(X_train, Y_train):
    """
    Learn the transformation matrix W* that maps X_train to Y_train.
    :returns: W* - a float matrix of shape [emb_dim x emb_dim]
    """
    # Compute W* using the least squares method
    W_star = np.linalg.lstsq(X_train, Y_train, rcond=None)[0]
    return W_star

# Calculate W
W = learn_transform(X_train, Y_train)

In [60]:
W = learn_transform(X_train, Y_train)

In [61]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8541285991668701),
 ('июнь', 0.8411203622817993),
 ('март', 0.839699387550354),
 ('сентябрь', 0.8359870314598083),
 ('февраль', 0.832929790019989),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278924226760864),
 ('июль', 0.823452889919281),
 ('август', 0.8120501637458801),
 ('декабрь', 0.803900420665741)]

In [62]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 300 is different from 0)

## UK-RU Translator

Now we are ready to make simple word-based translator: for each word in source language in shared embedding space we find the nearest in target language.


In [63]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [64]:
def translate(sentence):
    """
    Translate a sentence from Ukrainian to Russian.
    """
    words = sentence.split()
    translated_words = []
    for word in words:
        if word in uk_emb:
            # Map the word embedding
            mapped_vector = np.matmul(uk_emb[word], W)
            # Find the closest Russian word
            similar_words = ru_emb.similar_by_vector(mapped_vector, topn=1)
            translated_words.append(similar_words[0][0])
        else:
            translated_words.append(word)  # Keep the word as is if not in embeddings
    return " ".join(translated_words)


In [65]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

AssertionError: 

In [66]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: р›рёсѓрёс‡рєр° - сѓрµсѓс‚сђрёс‡рєр° с– рірѕрірє - рїр°рѕс–р±сђр°с‚
dst: р›рёсѓрёс‡рєр° – сѓрµсѓс‚сђрёс‡рєр° с– рірѕрірє – рїр°рѕс–р±сђр°с‚

src: рїрє р±сѓр»р° сѓрѕр±с– р»рёсѓрёс‡рєр° с‚р° р·сђрѕр±рёр»р° с…р°с‚рєсѓ, с‚р° р№ р¶рёрірµ. рђ с†рµ рїсђрёс…рѕрґсџс‚сњ с…рѕр»рѕрґрё. рћс‚ р»рёсѓрёс‡рєр° р·р°рјрµсђр·р»р° с‚р° р№ рїрѕр±с–рір»р° рі сѓрµр»рѕ рірѕрірѕсћ рґрѕр±сѓрір°с‚сњ, с‰рѕр± рірёс‚рѕрїрёс‚рё. рџсђрёр±с–рір°с” рґрѕ рѕрґрѕрѕс— р±р°р±рё с‚р° р№ рєр°р¶рµ:
dst: рїрє р±сѓр»р° сѓрѕр±с– р»рёсѓрёс‡рєр° с‚р° р·сђрѕр±рёр»р° с…р°с‚рєсѓ, с‚р° р№ р¶рёрірµ. рђ с†рµ рїсђрёс…рѕрґсџс‚сњ с…рѕр»рѕрґрё. рћс‚ р»рёсѓрёс‡рєр° р·р°рјрµсђр·р»р° с‚р° р№ рїрѕр±с–рір»р° пя сѓрµр»рѕ рірѕрірѕсћ рґрѕр±сѓрір°с‚сњ, с‰рѕр± рірёс‚рѕрїрёс‚рё. рџсђрёр±с–рір°с” рґрѕ рѕрґрѕрѕс— р±р°р±рё с‚р° р№ рєр°р¶рµ:

src: вђ” р—рґрѕсђрѕріс– р±сѓр»рё, р±р°р±сѓсѓсћ! р— рѕрµрґс–р»рµсћ... рџрѕр·рёс‡с‚рµ рјрµрѕс– рѕрірѕсћ, сџ рір°рј рѕрґсѓр»сѓр¶сѓ.
dst: вђ” р—рґрѕсђрѕріс– р±сѓр»рё, р±р°р±сѓсѓсћ! р— рѕрµрґс–р»рµсћ... рџрѕр·рёс‡с‚рµ р

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## 2 Task)  7 points

###  Download embedding from here: https://fasttext.cc/docs/en/crawl-vectors.html (download text format). Depending on the group, choose the correct Slavic language embedding. After downloading and using the embedding, repeat all the tasks from first part, but you dont need set asserts in this task and generate the same simple text (at least 200 words) and translate it into Russian.


## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -